In [7]:
"""
TorchGeo + U-Net (SMP) for continuous per-pixel regression (pixelwise regression)
- Inputs: multi-band georasters (e.g., pop, GDP, land-use)
- Labels: single-band continuous target in [0,1] (e.g., CISI), float32
- Pairing: IntersectionDataset (by spatial overlap)
- Loss: masked Huber (handles NoData via NaN)
- Mixed precision + grad clipping

Install:
    pip install torch torchvision torchgeo rasterio segmentation-models-pytorch albumentations
"""

import argparse
from pathlib import Path
import math
import warnings

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# -----------------------------
# TorchGeo imports (version-robust)
# -----------------------------
from torchgeo.datasets import RasterDataset

# stack_samples import moved across versions; try new then old
try:
    from torchgeo.datasets import stack_samples  # newer
except Exception:
    try:
        from torchgeo.datasets.utils import stack_samples  # older
    except Exception as e:
        raise ImportError("Could not import stack_samples from TorchGeo.") from e

# IntersectionDataset location varies slightly across versions
try:
    from torchgeo.datasets.geo import IntersectionDataset
except Exception:
    try:
        from torchgeo.datasets import IntersectionDataset
    except Exception as e:
        raise ImportError("Could not import IntersectionDataset from TorchGeo.") from e

from torchgeo.samplers import GridGeoSampler, RandomGeoSampler

# -----------------------------
# Model (U-Net from SMP)
# -----------------------------
import segmentation_models_pytorch as smp

warnings.filterwarnings("ignore", category=UserWarning)


# -----------------------------
# Dataset definitions (TorchGeo)
# -----------------------------
class InputsDataset(RasterDataset):
    """
    Multi-band inputs (e.g., population, GDP, land-use) as aligned GeoTIFFs.
    """
    filename_glob = "*.tif"
    is_image = True  # tensors will be under key "image"


class LabelsDataset(RasterDataset):
    """
    Single-band continuous target (e.g., CISI scaled to [0,1]).
    IMPORTANT:
      - use float32 for continuous targets
      - set is_image=False so TorchGeo stores this under key "mask"
      - store NoData as NaN in your GeoTIFFs when possible
    """
    filename_glob = "*.tif"
    is_image = False          # tensors will be under key "mask"
    dtype = torch.float32


# -----------------------------
# Normalization module
# -----------------------------
class ChannelWiseNormalize(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        m = torch.as_tensor(mean).view(1, -1, 1, 1)
        s = torch.as_tensor(std).view(1, -1, 1, 1)
        self.register_buffer("mean", m.float())
        self.register_buffer("std", s.float())

    def forward(self, x):
        return (x - self.mean) / (self.std + 1e-6)


# -----------------------------
# Masked Huber regression loss
# -----------------------------
class MaskedHuber(nn.Module):
    def __init__(self, delta=0.5):
        super().__init__()
        self.delta = delta

    def forward(self, pred, target, mask=None):
        """
        pred, target: (B,1,H,W)
        mask: (B,1,H,W) boolean/float in {0,1}. If None, uses isfinite(target).
        """
        if mask is None:
            mask = torch.isfinite(target)
        mask = mask.float()

        diff = pred - target
        abs_diff = diff.abs()
        delta = torch.tensor(self.delta, device=pred.device)
        quadratic = torch.minimum(abs_diff, delta)
        loss = 0.5 * quadratic**2 + delta * (abs_diff - quadratic) - 0.5 * (delta**2)
        loss = loss * mask

        denom = mask.sum().clamp_min(1.0)
        return loss.sum() / denom


# -----------------------------
# Training / validation driver
# -----------------------------
def run(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = True

    # ---- Build base datasets (positional root arg)
    inputs_ds = InputsDataset(args.in_dir)
    labels_ds = LabelsDataset(args.lab_dir)

    # ---- Combine by spatial overlap (modern replacement for ZipDataset)
    dataset = IntersectionDataset(inputs_ds, labels_ds)   # equivalently: inputs_ds & labels_ds

    # ---- Samplers
    train_sampler = RandomGeoSampler(dataset, size=args.patch, length=args.train_windows)
    val_sampler = GridGeoSampler(dataset, size=args.patch, stride=int(args.patch * args.val_stride_frac))

    # ---- DataLoaders (use TorchGeo collate)
    train_loader = DataLoader(
        dataset, batch_size=args.batch, sampler=train_sampler, num_workers=args.workers,
        collate_fn=stack_samples, pin_memory=True, persistent_workers=args.workers > 0
    )
    val_loader = DataLoader(
        dataset, batch_size=args.batch, sampler=val_sampler, num_workers=args.workers,
        collate_fn=stack_samples, pin_memory=True, persistent_workers=args.workers > 0
    )

    # ---- Peek for channel count and sanity
    sample = next(iter(train_loader))
    x0 = sample[0]["image"]      # (B,C,H,W)
    y0 = sample[1]["mask"]       # (B,1,H,W)
    in_ch = x0.shape[1]
    print(f"[Info] Inferred input channels: {in_ch}")
    print(f"[Info] Label shape sample: {tuple(y0.shape)} (expect B,1,H,W)")

    # ---- Normalization
    if args.use_quick_norm:
        mean, std = quick_channel_stats(train_loader, in_ch, args.norm_batches)
    else:
        mean = [0.0] * in_ch
        std = [1.0] * in_ch
    print(f"[Info] Normalization mean: {mean}")
    print(f"[Info] Normalization std : {std}")

    normalize = ChannelWiseNormalize(mean, std).to(device)

    # ---- Model: SMP U-Net
    model = smp.Unet(
        encoder_name=args.backbone,     # e.g., "resnet34"
        encoder_weights=None,           # use "imagenet" only if inputs are truly RGB-like
        in_channels=in_ch,
        classes=1,                      # single continuous target channel
        activation=None,                # handle activation explicitly
    ).to(device)

    # ---- Optimizer, loss, AMP
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.wd)
    criterion = MaskedHuber(delta=args.huber_delta)
    scaler = torch.cuda.amp.GradScaler(enabled=args.amp)

    # ---- Training loop
    best_val = math.inf
    for epoch in range(1, args.epochs + 1):
        model.train()
        tr_loss, tr_n = 0.0, 0
        for batch in train_loader:
            x = batch[0]["image"].float().to(device)  # (B,C,H,W)
            y = batch[1]["mask"].float().to(device)   # (B,1,H,W) in [0,1]; NaN = NoData preferred

            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=args.amp):
                x = normalize(x)
                yhat = model(x)
                yhat = torch.sigmoid(yhat)  # constrain to [0,1]
                loss = criterion(yhat, y)   # masked inside (NaNs ignored)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            tr_loss += loss.item()
            tr_n += 1

        # ---- Validation
        model.eval()
        va_loss, va_n = 0.0, 0
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=args.amp):
            for batch in val_loader:
                x = batch[0]["image"].float().to(device)
                y = batch[1]["mask"].float().to(device)
                x = normalize(x)
                yhat = torch.sigmoid(model(x))
                loss = criterion(yhat, y)
                va_loss += loss.item()
                va_n += 1

        tr_loss /= max(tr_n, 1)
        va_loss /= max(va_n, 1)
        print(f"Epoch {epoch:03d} | train: {tr_loss:.4f} | val: {va_loss:.4f}")

        if va_loss < best_val:
            best_val = va_loss
            ckpt = {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "norm_mean": mean,
                "norm_std": std,
                "in_channels": in_ch,
                "backbone": args.backbone,
            }
            Path(args.out_dir).mkdir(parents=True, exist_ok=True)
            torch.save(ckpt, Path(args.out_dir) / "best_unet_regression.pt")
            print(f"  -> saved best checkpoint (val={best_val:.4f})")

    print("Done.")


def quick_channel_stats(train_loader, in_ch, norm_batches):
    """Rough channel-wise mean/std over a few batches."""
    running_mean = torch.zeros(in_ch, dtype=torch.float64)
    running_var = torch.zeros(in_ch, dtype=torch.float64)
    n = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            xb = batch[0]["image"].float()  # (B,C,H,W)
            B, C, H, W = xb.shape
            xb = xb.view(B, C, -1)
            running_mean += xb.mean(dim=(0, 2)).double()
            running_var += xb.var(dim=(0, 2), unbiased=False).double()
            n += 1
            if i + 1 >= norm_batches:
                break
    mean = (running_mean / max(n, 1)).float().tolist()
    std = (running_var / max(n, 1)).sqrt().float().tolist()
    return mean, std


# -----------------------------
# CLI
# -----------------------------
def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--in_dir", type=str, required=True, help="Path to inputs directory (multi-band GeoTIFFs)")
    p.add_argument("--lab_dir", type=str, required=True, help="Path to labels directory (single-band GeoTIFFs)")
    p.add_argument("--out_dir", type=str, default="checkpoints", help="Where to save checkpoints")

    p.add_argument("--patch", type=int, default=256, help="Patch size in pixels (H=W)")
    p.add_argument("--train_windows", type=int, default=2000, help="#random windows per epoch for training")
    p.add_argument("--val_stride_frac", type=float, default=1.0, help="Val stride as fraction of patch size (>=1)")
    p.add_argument("--batch", type=int, default=4)
    p.add_argument("--workers", type=int, default=4)

    p.add_argument("--epochs", type=int, default=50)
    p.add_argument("--lr", type=float, default=3e-4)
    p.add_argument("--wd", type=float, default=1e-2)
    p.add_argument("--huber_delta", type=float, default=0.5)
    p.add_argument("--backbone", type=str, default="resnet34")  # any SMP encoder
    p.add_argument("--amp", action="store_true", help="Enable mixed precision")

    p.add_argument("--use_quick_norm", action="store_true", help="Estimate mean/std over a few batches")
    p.add_argument("--norm_batches", type=int, default=10, help="#batches for quick norm estimation")
    return p.parse_args()


if __name__ == "__main__":
    class Args: pass
    args = Args()
    args.in_dir = "data/inputs"
    args.lab_dir = "data/labels"
    args.out_dir = "checkpoints"

    args.patch = 256
    args.train_windows = 500
    args.val_stride_frac = 1.0
    args.batch = 4
    args.workers = 2

    args.epochs = 5
    args.lr = 3e-4
    args.wd = 1e-2
    args.huber_delta = 0.5
    args.backbone = "resnet34"
    args.amp = True

    args.use_quick_norm = True
    args.norm_batches = 5

    run(args)


DatasetNotFoundError: Dataset not found in `paths='data/inputs'` and cannot be automatically downloaded, either specify a different `paths` or manually download the dataset.

In [8]:
from pathlib import Path
from glob import glob

print("inputs exists? ", Path("data/inputs").exists())
print("labels exists? ", Path("data/labels").exists())
print("# *.tif in inputs:", len(glob("data/inputs/*.tif")))
print("# **/*.tif in inputs:", len(glob("data/inputs/**/*.tif", recursive=True)))
print("# *.tiff in inputs:", len(glob("data/inputs/*.tiff")))
print("# **/*.tiff in inputs:", len(glob("data/inputs/**/*.tiff", recursive=True)))

print("# **/*.tif in labels:", len(glob("data/labels/**/*.tif", recursive=True)))
print("# **/*.tiff in labels:", len(glob("data/labels/**/*.tiff", recursive=True)))

inputs exists?  False
labels exists?  False
# *.tif in inputs: 0
# **/*.tif in inputs: 0
# *.tiff in inputs: 0
# **/*.tiff in inputs: 0
# **/*.tif in labels: 0
# **/*.tiff in labels: 0
